<a href="https://colab.research.google.com/github/JulioHaro93/PracticasTesisKeras/blob/main/aprendizaje_por_transferencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install tensorflow==2.12

Instalamos tqdm

In [33]:
!pip install tqdm

Descargamos el dataset desde el ambiente de desarrollo de google

In [34]:
!wget --no-check-certificate \https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \-O ./cats_and_dogs_filtered.zip

--2025-05-07 00:09:21--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.115.207, 172.253.122.207, 172.253.63.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.115.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  84.2MB/s    in 0.8s    

2025-05-07 00:09:22 (84.2 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



Preprocesamiento de datos

In [35]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator #OJO AQUÍ

%matplotlib inline
tf.__version__

'2.12.0'

Descomprimir dataset de perros vs Gatos

In [36]:
dataser_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataser_path, mode="r")

zip_object.extractall("./")
zip_object.close()

**Es importante automatizar el preprocesado con la gfunción de ImageDataGenerator, eso les da el tamaño y la forma homogénea**

In [37]:
dataser_path_new = "./cats_and_dogs_filtered/"

train_dir = os.path.join(dataser_path_new, "train")
validation_dir = os.path.join(dataser_path_new, "validation")

# **Construcción del modelo**

Primero cargamos el modelo pre entrenado, en este caso (MobileNetV2)

El modelo depende del tipo de imagen, por lo que lo esperado es la salida la tenemos que modificar nosotors, por eso en este caso se utiliza include_top = False para que no sea la capa final de la cabecera-

In [38]:
IMG_SHAPE = (128,128,3)

#Aquí usamos MobileNetV2 porque inception tardaría mucho tiempo para entrenar en esta clase

base_model= tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")


In [39]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

Cargamos el modelo pre entrenado, primero congelamos la base

In [40]:
base_model.trainable = False

Definir la cabecera personalizada para nuestra red neuronal

In [41]:
base_model.output #me permite ver el número de salidas y qué sale en el modelo base
#en este caso es muy grande

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

Establecemos las siguientes capas

Siempre se diseña con forma al tipo de clasificación que vamos a realizar, el ejemplo es el tamaño de salida que nos ha devuelto la red. En este caso, para ete dataset es binaria

In [43]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d_3')>

In [44]:
9prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

definición del modelo

In [45]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_2[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

Compilar el modelo

In [46]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001), loss="binary_crossentropy", metrics=["accuracy"])

Crear generadores de datos
Las grandes arquitecturas pre etrenadas sólo soportan cierto tipo de tamaños de imágenes.

Para MobilNet soporta: (96,96) (128,128), (160, 160), (192,192),(224,224)
* Nota todas son cuadradas, supongome yo que para mejorar el producto matricial al momento de multiplicar por pesos y sumar bias

In [49]:
#Usamos ImageDataGenerator para generar los datos dividiendo entre 255 y reescalamos

data_gen_train= ImageDataGenerator(rescale= 1/255 )
data_gen_valid = ImageDataGenerator(rescale=1/255)

In [50]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [54]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size = (128, 128), batch_size = 128, class_mode="binary")

Found 1000 images belonging to 2 classes.


In [55]:
model.fit(train_generator, epochs=10, validation_data= valid_generator)

Epoch 1/10
16/16 [==============================] - 47s 3s/step - loss: 0.7821 - accuracy: 0.5135 - val_loss: 0.7801 - val_accuracy: 0.5010
Epoch 2/10
16/16 [==============================] - 50s 3s/step - loss: 0.7672 - accuracy: 0.5245 - val_loss: 0.7672 - val_accuracy: 0.5070
Epoch 3/10
16/16 [==============================] - 50s 3s/step - loss: 0.7545 - accuracy: 0.5335 - val_loss: 0.7550 - val_accuracy: 0.5250
Epoch 4/10
16/16 [==============================] - 46s 3s/step - loss: 0.7428 - accuracy: 0.5435 - val_loss: 0.7433 - val_accuracy: 0.5340
Epoch 5/10
16/16 [==============================] - 49s 3s/step - loss: 0.7310 - accuracy: 0.5540 - val_loss: 0.7320 - val_accuracy: 0.5420
Epoch 6/10
16/16 [==============================] - 50s 3s/step - loss: 0.7199 - accuracy: 0.5645 - val_loss: 0.7208 - val_accuracy: 0.5580
Epoch 7/10
16/16 [==============================] - 53s 3s/step - loss: 0.7084 - accuracy: 0.5790 - val_loss: 0.7096 - val_accuracy: 0.5650
Epoch 8/10
16/16 [==

Evaluar el modelo de aprendizaje por transferencia

se usa el valid_generator porque genera la presición cuando se utiliza un image_generator, esto es importante para la tesis



In [57]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

<ipython-input-57-9c01a3b88477>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


Accuracy after transfer learning: 0.5920000076293945


**Aquí en adelante se presentan técnicas que mejoran el accuracy**

Técnica puesta a punto (fine tuning) - NUNCA ENTRENAR TODA LA RED, Sólo algunas capas

Considerar que se usa después de hacer un transfer learning, en muchos casos mejora los resultados de clasificación

Primero se congelan y después se hace el fine tuning, es decir, se entrena al menos una vez, y después ya se hace el fine tuning

In [56]:
#primero decongelamos el modelo base
base_model.trainable = True

print("número de capas en el modelo base {}".format(len(base_model.layers)))


número de capas en el modelo base 154


Elejimos las capas para el fine tuning

In [58]:
fine_tune_at = 100

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [59]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001), loss="binary_crossentropy", metrics=["accuracy"])

In [60]:
model.fit(train_generator, epochs=5, validation_data= valid_generator)

Epoch 1/5
16/16 [==============================] - 72s 4s/step - loss: 0.5281 - accuracy: 0.7395 - val_loss: 0.3715 - val_accuracy: 0.8600
Epoch 2/5
16/16 [==============================] - 63s 4s/step - loss: 0.3552 - accuracy: 0.8745 - val_loss: 0.2786 - val_accuracy: 0.8970
Epoch 3/5
16/16 [==============================] - 65s 4s/step - loss: 0.2635 - accuracy: 0.9260 - val_loss: 0.2242 - val_accuracy: 0.9260
Epoch 4/5
16/16 [==============================] - 69s 4s/step - loss: 0.2048 - accuracy: 0.9475 - val_loss: 0.1884 - val_accuracy: 0.9370
Epoch 5/5
16/16 [==============================] - 64s 4s/step - loss: 0.1657 - accuracy: 0.9565 - val_loss: 0.1641 - val_accuracy: 0.9450


El fine tuning funciona adecuadamente con bases de datos muy grandes.

In [61]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

<ipython-input-61-9c01a3b88477>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)


Accuracy after transfer learning: 0.9449999928474426
